# Water flow in soil - Numerical solutions in DuMu${}^x$

_by Daniel Leitner, Timo Koch, Andrea Schnepf_

In the following we will give some hints how to solve Benchmarks 1-4, using [DuMu${}^x$](www.dumux.org). Scripts for the following benchmarking examples are available on [Github](https://github.com/Plant-Root-Soil-Interactions-Modelling/dumux-rosi.git), in the folder dumux-rosi/rosi_benchmarking/soil/ 

For 1D run /python/dumux_1d\*.py, for 3D /python/dumux_3d\*.py, for unstructured grids /python/dumux_ug\*.py.

To run the examples you will need: dune (releases/2.6), dune-foamgrid (releases/2.6), dumux (releases/3.0), CRootBox (master)


## Theory

We base our Dumux implementation on the [Richards Problem](http://www.dumux.uni-stuttgart.de/doxygen-stable/html-2.12/a01911.php) which states the Richards equation in the following form:
\begin{eqnarray} 
\frac{\partial}{\partial t} \left( \phi S \rho \right) - \frac{\partial}{\partial z}  \left[\rho \frac{\kappa}{\mu} K_s \left(\frac{\partial \psi}{\partial z} - \rho g \right) \right] = 0,
\end{eqnarray}
where $\rho$ is the fluid density, $S$ is the fluid saturation, $\phi$ is the porosity, $\kappa$ is the relative permeability, $\mu$ the dynamic viscosity, $K_s$ the intrinsic permeability, $\psi$ the fluid pressure and $g$ the gravitational acceleration. The quantities $S$ and $\psi$ are related by the water retention curve, i.e. $S=f^{-1}(\psi_{ref}-\psi) $

Introducing the hydraulic conductivity $K_c = K \ \rho \ g / \mu$, with $K = \kappa \ K_s$, and the water content $\theta = \phi \ S$ the equation takes the form 
\begin{eqnarray} 
\frac{\partial}{\partial t} \left( \rho \theta \right) = \frac{\partial}{\partial z}  \left[ K_c \frac{1}{g} \left(\frac{\partial \psi}{\partial z} - \rho g \right) \right].
\end{eqnarray}

Using the pressure head $h = (\psi-\psi_{ref})/(\rho \ g)$ leads to
\begin{eqnarray} 
\frac{\partial}{\partial t} \left( \rho \theta \right) = \frac{\partial}{\partial z}  \left[ \rho K_c \left(\frac{\partial h}{\partial z} - 1 \right) \right],
\end{eqnarray}
and dividing the equation by $\rho$ leads to the Richards equation, as presented in [Water flow in soil](Soil water flow.ipynb) (the fluid density $\rho$ must be constant in space and time). 

## Compile program

We compile the program by calling 

*make richards1d* (for 1D)

*make richards3d* (for 3D)

*make richardsUG* (for unstructured grids)

in the folder dumux-rosi/build-cmake/rosi_benchmarking/soil.

## Run benchmarks
All benchmarks can be simulated by just changing the Dumux .input files. The specific benchmarks can be started by Python scripts located in the folder dumux-rosi/build-cmake/rosi_benchmarking/soil/python

## Input files

The input files for benchmarks 1-4 are located in /soil/benchmarks_1d/ (for 1D), /soil/benchmarks_3d/ (for 3D), and /soil/benchmarks_ug/ (for unstructured grids). The following paramteters are used:

**[Problem]**

*Name*, name of the simulation

**[Grid]**

*UpperRight*, in 1d actually just the upper coordinate (per default = 0, in all benchmarks) [m]

*LowerLeft*, lower coordinate [m]

*Cells*, number of cells (degrees of freedom *dof*-1 for discretization with box method) 

or

*File*, name of the 1d dune grid file (.dgf)

**[ TimeLoop ]**

*TEnd*, simulation time [s]. If the value is zero a steady state problem is calculated

*DtInitial*, initial time step [s]

*MaxTimeStepSize*, maximal time step size [s]

*CheckTimes*, optionally times [s] can be defined, when the .vtp output is written. Per default a file at start and at the final time is written.

**[Soil.BC.Top]**

*Type*, type of the top boundary condition (1 = dirichlet, 2 = neumann, 4 = atmospheric, see [Climate])

*Value*, [cm] pressure head for dirchlet (type 1), and [cm/day] for neumann or atmospheric (type 2 & 4) 

**[Soil.BC.Bot]**

*Type*, type of the top boundary condition (1 = dirichlet, 2 = neumann, 5 = free drainage)

*Value*, [cm] pressure head for dirchlet (type 1), and [cm/day] for neumann (type 2). 

**[Soil.VanGenuchten]**

The set of Van Genuchten parameters: *Qr* [1], *Qs* [1], *Alpha* [1/cm], *N* [1], and *Ks* [m/s]. If there are multiple soil layers, each parameter is given by multiple values (i.e. one value per soil layer). 

**[Soil.IC]**

*Pressure*, if it is one value: constant pressure [cm] pressure head is assumed. If there are more values, and additionally, *Z* values are defined, tabular linear interpolated data is assumed . If there are more values, but no *Z* values, the pressure is assumed to be given constant per soil layer.

*Z*, optionally, ascending values are given for linear interpolation, sample points are (*Z*, *Pressure*) [m] 

**[Soil.Layer]**

*Number*, optionally, multiple soil layers can be defined. Either by tabular data (linear interpolated and rounded to integer values) if *Z* values are defined. Or, the layer number is given within the grid file as first parameter. In this case we do not need to set the [Soil.Layer] tag, but presence of grid data is indicated by multiple VanGenuchten parameters.

*Z*, optionally, ascending values are given for linear interpolation, sample points are (*Z*, *Number*) [m] 

**[Climate]**

*Precipitation*, negative value is precipitation [kg/(s m²)]. Given by a linear table. Implemented with surface run-off, maximal pressure is -10000 [cm]. 

*Time*, ascending values , sample points are (*Time*, *Precipitation*) [s]


## Remarks

### Benchmark 1 

Benchmark 1 has multiple (two) sets of Van Genuchten parameters. The soil layer is given by linear interpolation based on the z-coordinate within the soil.

When calculating steady state it is sufficient to set *TimeLoop.TEnd* to zero. But sometimes the solution is not found. In this case either choose better initial conditions (which is used as initial guess), or set *TimeLoop.TEnd* to a very large value. 

In 1D we chose a large value, in 3D we picked a good guess and calculated the steady state example. 

For unstructured grids we only calculate steady state for subplot (c), but choose a large value for (a) & (b). Even when calculating in parallel the unstructured solution takes some time.

### Benchmark 2 

Initial condition is given by linear interpolation. All benchmarks are calculated as steady state problems.

### Benchmark 3

The output times are defined with TimeLoop.CheckTimes.

Note that for top boundary condition we use atmospheric, and give the precipitation in the [Climate] tag. In this way the boundary condition ensures [?????].

For 3D examples the number of CPU cores that are used can be defined in the Python script.


### Benchmark 4 

Again, we choose atmospheric boundary conditions as top boundary condition.

When we choose atomsopheric boundary conditions additionally a .csv file ist written, containing two values per line: simulation time [s] and actual precipitation [kg/(m²\*s)]. We read this file for visualization of the results.


## Implementation

For implementation of the Benchmarks 1-4, we have to convert the parameters, overwrite the problem class defining initial conditions and boundary conditions, and manage the parameters using an input file. In the following we describe the most relevant C++ code pieces 


### main file: richards.cc 

Compilation with make richards1d, richards3d, and richardsUG compiles this file with different grid types, defined as variable GRIDTYPE in the CMakeLists.txt file. It is a normal Dumux 3.0 main file, with extensions:

1. The TimeLoop.CheckTimes are set in L108-124.

2. We check if the example is steady state or dynamic and calculate it accordingly. 

3. We only write vtk files at the check times L165-167.



### problem file: richardsproblem.hh

A Dumux Problem class describes initial conditions and boundary conditions of the problem. 

The constructor reads all relevant parameters from the input file, in our case: Soil.BC.Top.Type, Soil.BC.Top.Value, Soil.BC.Bot.Type, Soil.BC.Bot.Value, Climate.Precipitation, Climate.Time, Soil.IC.P, Soil.IC.Z. Linear interpolation is handled by the InputFileFunction class in dumux-rosi/dumux/io/inputfilefunction.hh.

Definition of boundary conditions:

Definition of initial condition: 


### spatial parameters: richardsparams.hh

This class manages the spatial parameters. 


